# Drought analyser

#### The algorithm
This function is used to detect and analyse droughts. As input it needs discharge data (as a dataframe), a basin name (or the column name of the discharge) and a critical water flow. Then, the algorithm checks how many times the flow goes underneath the threshold, counts the amount of days of drought and the maximum amount of water that is lost during this drought. This file is used so the function does not need to be defined in every file.

In [10]:
def drought_analyser(df, basin_name: str, q_crit: float):
    droughts = []
    drought_counter = 0
    i = 0
    
    while i < len(df):
        if df.iloc[i] <= q_crit:
            drought_counter += 1
            days_counter = 0
            deficit_list = []
            cum_def_list = []
            accum_deficit = 0
            max_cumulative_deficit = 0
            start_date = df.index[i]
            
            while i < len(df):
                deficit = df.iloc[i] - q_crit
                deficit_list.append(deficit)
                accum_deficit += deficit
                cum_def_list.append(accum_deficit)
                max_cumulative_deficit = min(max_cumulative_deficit, accum_deficit)
                days_counter += 1
                i += 1
                
                if accum_deficit > 0:
                    break
            
            if max_cumulative_deficit < 0:
                droughts.append({
                    "Drought Number": drought_counter,
                    "Start Date": start_date,
                    "Duration (days)": days_counter,
                    "Max Cumulative Deficit (m3/s)": max_cumulative_deficit,
                    "Cum Deficit List": cum_def_list
                })
        else:
            i += 1
    if drought_counter < 1:
        droughts.append({
                "Drought Number": 0,
                "Start Date": df.index[0],
                "Duration (days)": 0,
                "Max Cumulative Deficit (m3/s)": 0,
                "Cum Deficit List": 0
            })
        
    return pd.DataFrame(droughts)

#### Other re-occuring functions

In [2]:
def convert_Qsim_mmday_to_m3s(Q_sim_mmday, area_km2):
    return (Q_sim_mmday * area_km2) / 86.4

In [12]:
def earth_movers_distance(x_obs, y_obs, x_model, y_model):
    duration_emd = wasserstein_distance(x_obs, x_model)
    deficit_emd = wasserstein_distance(y_obs, y_model)
    total_distance = duration_emd * 0.5 + deficit_emd * 0.5

    return duration_emd * 0.5, deficit_emd * 0.5